In [ ]:
"Installation des Packages"
!pip install openpyxl
!pip install pandas fiona shapely pyproj rtree # à faire obligatoirement en premier pour utiliser rtree ou pygeos pour les jointures spatiales
!pip install geopandas
!pip install contextily
!pip install folium

In [ ]:
"Import des librairies"
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
"Import et lecture de la bdd"
df=pd.read_excel('/Users/aharonelbez/Desktop/Projet python 1/Suisse/suissedata1.xlsx')
df=df.drop(columns=['Remarque','Carte','Carte.1','N° sv.85','py','N° sv.','Cc','PE','PT','N° ET','Sigle ET','N° GO','Sigle GO','Nom long','Sigle sv.','PC','PP','ST'])
df.head()


#On recupère que les données concernant les trains
df1=df.copy()

df1=df1[df1['Moyen de transport']=='Zug']
df1.head()




In [ ]:
#1. On crée un geodf

map_suisse= gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['Coord. E'], df1['Coord. N']))
map_suisse.head()
map_suisse.crs


# Tentative de mettre un fond de cartes 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world
ax1 = world[world['name'] == 'Switzerland'].plot(
    color='white', edgecolor='black')

map_suisse.plot(alpha=0.2,edgecolor='black',figsize=(20,20))

plt.show()


In [ ]:
#2eme option avec folium
 
import folium
print(folium.__version__)

kanton_map = folium.Map(location=[46.8, 8.33],
                   tiles='OpenStreetMap', zoom_start=7)
kanton_map.choropleth(geo_data='/Users/aharonelbez/Desktop/Projet python/mapswtzerland.geojson')
kanton_map


import numpy as np

stations = list()
for station in df1['Nom (ordre alphab.)']:
    stations+= [df1.geometry[df1['Nom (ordre alphab.)']==station]]
stations = [item for sublist in stations for item in sublist]
print(stations[:3])

x=np.array([1,2,3,4,5,6,7,8,9,10])

def peer(x):
    y=[]
    z=[]
    for i in range(0,len(x)-1,2):
        y=[x[i],x[i+1]]
        z+=[y]
    return z
peer(x)

stations2=peer(stations)
print(stations2[:4])


#-1er option
feature_group = folium.FeatureGroup("Locations")
for i in range(0,len(stations),2):
    feature_group.add_child(folium.Marker(location=[stations[i],station[i+1]]))
 
#-2eme option   
feature_group = folium.FeatureGroup("Locations")
for station in stations:
    feature_group.add_child(folium.Marker(location=[station[0], station[1]])) 
kanton_map.add_child(feature_group)
kanton_map
 
kanton_map.add_child(feature_group)
kanton_map


In [ ]:
#3eme option fichier geojson

df_map=gpd.read_file('mapswtzerland.geojson')

df_map.head()

import geopandas as gpd
df_test = gpd.read_file('mapswtzerland.geojson')
print(df_test.columns)
df_test.head()
df_test.plot()

In [ ]:
#4eme option avec OpenStreetMap
m = folium.Map(location=[46.8, 8.33], tiles="OpenStreetMap", zoom_start=7)

for i in range(0,len(df1)):
       folium.Marker(location=[df1.iloc[i]['Coord. E'], df1.iloc[i]['Coord. N']],popup=df1.iloc[i]['Nom (ordre alphab.)']).add_to(m)

m
